In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import tensorflow as tf
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import re
from sklearn import preprocessing
from tqdm import tqdm
import scipy.sparse

import tensorflow as tf
import tensorflow_hub as hub

In [12]:
df=pd.read_csv("../complaints.csv")

/var/folders/k4/29bygr_s3mv0rn4yh7_rvtmm0000gn/T/ipykernel_5234/2118662780.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../complaints.csv")


In [13]:
df.head(3)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2018-07-31,Checking or savings account,Savings account,Opening an account,Unable to open an account,NaN,Company believes the complaint is the result o...,"Bank of CFPB, Washington, DC CFPB PIV",NaN,NaN,NaN,NaN,Web,2018-07-31,Closed with explanation,Yes,NaN,3527315
1,2020-10-27,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,HEARTLAND PAYMENT SYSTEMS INC,NJ,7060.0,NaN,NaN,Fax,2020-10-27,Closed with explanation,Yes,NaN,4491254
2,2018-08-02,Mortgage,Conventional home mortgage,Closing on a mortgage,NaN,NaN,Company believes the complaint is the result o...,"Bank of CFPB, Washington, DC CFPB PIV",NaN,NaN,NaN,NaN,Web,2018-08-02,Closed with explanation,Yes,NaN,3527438


In [14]:
# df=df[df['Consumer disputed?'].notna()]
# df=df.drop(["Complaint ID","Timely response?","Date sent to company","Tags","Submitted via","Company public response","Consumer consent provided?","Sub-issue"],axis=1)

In [15]:
# df.isna().sum()

In [16]:
COLN=['Date received','Product','Sub-product','Issue','Consumer complaint narrative','Company','State','ZIP code','Company response to consumer','Consumer disputed?']
ONE_HOT_ENC=['product','sub-product','issue', 'company', 'state','response']
FILL_MISSING=["sub-product","state","zip-code","narrative"]
def print_unique_values(df,coln):
    arr=np.unique(df[coln].to_list())
    for i in range(len(arr)):
        print(i,"--",arr[i])
    del arr
    
#pre processing function
def fill_in_missing(df,coln):
    df[coln]=df[coln].fillna("")
    return df
    

def transform_name(df):
    df=df[df['Consumer disputed?'].notna()]
    df=df.drop(["Complaint ID","Timely response?","Date sent to company","Tags","Submitted via","Company public response","Consumer consent provided?","Sub-issue"],axis=1)
    df.columns=['date-received', 'product', 'sub-product', 'issue','narrative', 'company', 'state', 'zip-code','response', 'disputed']
    return df

def one_hot_enc(df,coln):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(np.array(df[coln]))
    s=enc.transform(np.array(df[coln]))
    return s.toarray()

def binarize_target(x):
    lb = preprocessing.LabelBinarizer()
    lb.fit(['No','Yes'])
    return lb.transform(x)


def convert_zip_code(zip_code):
    if zip_code == '':
        zip_code = "00000"
    zip_code=re.sub("[^0-9]", "",zip_code)
    zip_code = re.sub(r'X{0,5}', "0", zip_code)
    zip_code = np.float32(zip_code)
    return zip_code


In [17]:
convert_zip_code("89-11")

80901010.0

In [18]:
# Credits https://www.kaggle.com/code/ashwinids/cleaning-exploring-consumer-complaints-data
mapping_old2new = {
    "Auto": "Auto debt",
    "Credit card": "Credit card debt",
    "Federal student loan": "Federal student loan debt",
    "Medical": "Medical debt",
    "Mortgage": "Mortgage debt",
    "Non-federal student loan": "Private student loan debt",
    "Other (i.e. phone, health club, etc.)": "Other debt",
    "Payday loan": "Payday loan debt",
    "Non-federal student loan": "Private student loan",
    "Federal student loan servicing": "Federal student loan",
    "Credit repair": "Credit repair services",
    "Credit reporting": "Credit reporting",
    "Conventional adjustable mortgage (ARM)": "Conventional home mortgage",
    "Conventional fixed mortgage": "Conventional home mortgage",
    "Home equity loan or line of credit": "Home equity loan or line of credit (HELOC)",
    "Other": "Other type of mortgage",
    "Other mortgage": "Other type of mortgage",
    "Second mortgage":"Other type of mortgage",
    "Credit card": "General-purpose credit card or charge card",
    "General purpose card": "General-purpose prepaid card",
    "Gift or merchant card": "Gift card",
    "Electronic Benefit Transfer / EBT card": "Government benefit card",
    "Government benefit payment card": "Government benefit card",
    "ID prepaid card": "Student prepaid card",
    "Other special purpose card":  "Other prepaid card",
    "Store credit card": "Other prepaid card",
    "Transit card": "Other prepaid card",
    "(CD) Certificate of deposit": "CD (Certificate of Deposit)",
    "Other bank product/service": "Other banking product or service",
    "Cashing a check without an account": "Other banking product or service",
    "Vehicle lease": "Lease",
    "Vehicle loan": "Loan",
    "Check cashing": "Check cashing service",
    "Mobile wallet": "Mobile or digital wallet",
    "Traveler’s/Cashier’s checks": "Traveler's check or cashier's check"
}
prod2sub = {
    "Auto debt": "Debt collection",
    "Credit card debt": "Debt collection",
    "Federal student loan debt": "Debt collection",
    "I do not know": "Debt collection",
    "Medical debt": "Debt collection",
    "Mortgage debt": "Debt collection",
    "Private student loan debt": "Debt collection",
    "Other debt": "Debt collection",
    "Payday loan debt": "Debt collection",
    "Credit repair services": "Credit reporting, credit repair services, or other personal consumer reports",
    "Credit reporting": "Credit reporting, credit repair services, or other personal consumer reports",
    "Other personal consumer report": "Credit reporting, credit repair services, or other personal consumer reports",
    "Conventional home mortgage": "Mortgage",
    "FHA mortgage": "Mortgage",
    "Home equity loan or line of credit (HELOC)": "Mortgage",
    "Other type of mortgage": "Mortgage",
    "Reverse mortgage": "Mortgage",
    "VA mortgage": "Mortgage",
    "General-purpose credit card or charge card": "Credit card or prepaid card",
    "General-purpose prepaid card": "Credit card or prepaid card",
    "Gift card": "Credit card or prepaid card",
    "Government benefit card": "Credit card or prepaid card",
    "Student prepaid card": "Credit card or prepaid card",
    "Payroll card": "Credit card or prepaid card",
    "Other prepaid card": "Credit card or prepaid card",
    "CD (Certificate of Deposit)": "Checking or savings account",
    "Checking account": "Checking or savings account",
    "Other banking product or service": "Checking or savings account",
    "Savings account": "Checking or savings account",
    "Lease": "Vehicle loan or lease",
    "Loan": "Vehicle loan or lease",
    "Federal student loan": "Student loan",
    "Private student loan": "Student loan",
    "Installment loan": "Payday loan, title loan, or personal loan",
    "Pawn loan": "Payday loan, title loan, or personal loan",
    "Payday loan": "Payday loan, title loan, or personal loan",
    "Personal line of credit": "Payday loan, title loan, or personal loan",
    "Title loan": "Payday loan, title loan, or personal loan",
    "Check cashing service": "Money transfer, virtual currency, or money service",
    "Debt settlement": "Money transfer, virtual currency, or money service",
    "Domestic (US) money transfer": "Money transfer, virtual currency, or money service",
    "Foreign currency exchange": "Money transfer, virtual currency, or money service",
    "International money transfer": "Money transfer, virtual currency, or money service",
    "Mobile or digital wallet": "Money transfer, virtual currency, or money service",
    "Money order": "Money transfer, virtual currency, or money service",
    "Refund anticipation check": "Money transfer, virtual currency, or money service",
    "Traveler's check or cashier's check": "Money transfer, virtual currency, or money service",
    "Virtual currency": "Money transfer, virtual currency, or money service",
    "Not given":"Not given"
}

# Credits https://www.kaggle.com/code/ashwinids/cleaning-exploring-consumer-complaints-data

def get_subprods(x):
    
    if x['sub-product'] in mapping_old2new:
        if x['sub-product']=="Other":
            if x['product']=='Mortage':
                return("Other type of mortgage")
            else:
                return("Other debt")
        else:
            return(mapping_old2new[x['sub-product']])
    else:
        return(x['sub-product'])



# Credits https://www.kaggle.com/code/ashwinids/cleaning-exploring-consumer-complaints-data

prodmap = {
    "Payday loan": "Payday loan, title loan, or personal loan",
    "Credit reporting": "Credit reporting, credit repair services, or other personal consumer reports",
    "Credit card": "Credit card or prepaid card"
}
def get_product(x):
    
    if not isinstance(x['sub-product'], str):
        if x['product'] in prodmap:
            return(prodmap[x['product']])
        else:
            return(x['product'])
    else:
        return(prod2sub[x['sub-product']])



In [19]:
def map_values(df):
    df['sub-product'] = df[['product','sub-product']].apply(lambda x: get_subprods(x), axis =1)
    df['product'] = df[['product','sub-product']].apply(lambda x: get_product(x), axis =1)
    return df

In [20]:
def preprocess(df):
    df=transform_name(df)
    df=map_values(df)
    df=fill_in_missing(df,FILL_MISSING)
    #time based splitting
    df=df.sort_values("date-received")
    train=df.iloc[0:int(np.floor(df.shape[0]*0.67)),:]
    test=df.iloc[int(np.floor(df.shape[0]*0.67)):,:]
    #time based splitting
    return train,test

    
    

def preprocess_2(df,filename):
    arr=[]
    lent=df.shape[0]
    zip_code=list(map(str,df['zip-code'].tolist()))
    for i in tqdm(range(0,lent)):
        arr.append(convert_zip_code(zip_code[i]))
    arr=np.array(arr)
    del zip_code
    
    frst=one_hot_enc(train,ONE_HOT_ENC)
    scnd=np.array(arr)
    del arr
    
    arr2=[]
    for i in tqdm(range(scnd.shape[0])):
        arr2.append(np.concatenate((frst[i],[scnd[i]]),axis=0))
    arr2=np.array(arr2)
    del frst 
    del scnd
    
    sparse_matrix=scipy.sparse.csc_matrix(np.array(arr2))
    del arr2
    np.save(filename,sparse_matrix)


In [21]:
train,test=preprocess(df)
# del train
del df

In [24]:
train.shape

(514866, 10)

In [25]:
test.shape

(253591, 10)

,date-received,product,sub-product,issue,narrative,company,state,zip-code,response,disputed
1282503,2016-01-23,Credit card or prepaid card,Government benefit card,Adding money,,"Empowerment Ventures, LLC",DE,19713,Closed with explanation,No
2610446,2016-01-24,Mortgage,Home equity loan or line of credit (HELOC),"Loan servicing, payments, escrow account",The bank filed to take my payment on XXXX resu...,JPMORGAN CHASE & CO.,NY,11249,Closed with explanation,Yes
342196,2016-01-24,Debt collection,Medical debt,Cont'd attempts collect debt not owed,,"Thunderbird Collection Specialists, Inc.",AZ,85122,Closed with explanation,No
1357629,2016-01-24,Mortgage,Conventional home mortgage,"Loan modification,collection,foreclosure",In XXXX XXXX I was offered a {$11000.00} cash ...,HSBC NORTH AMERICA HOLDINGS INC.,NY,11050,Closed with explanation,No
1351296,2016-01-24,"Credit reporting, credit repair services, or o...",,Incorrect information on credit report,I recently applied for an apartment in XXXX XX...,CORELOGIC INC,MI,48224,Closed with explanation,No
...,...,...,...,...,...,...,...,...,...,...
921587,2017-04-22,Checking or savings account,CD (Certificate of Deposit),"Account opening, closing, or management",Bank accounts ( CD & Money Markets ) w as ope...,U.S. BANCORP,IL,60643.0,Closed with explanation,No
1356884,2017-04-22,Credit card or prepaid card,,Delinquent account,I have dispute d my Capital One ac count many ...,CAPITAL ONE FINANCIAL CORPORATION,GA,30236,Closed with explanation,No
2684748,2017-04-22,Checking or savings account,Checking account,Using a debit or ATM card,,KEYCORP,OH,44136,Closed with explanation,No
527836,2017-04-22,"Credit reporting, credit repair services, or o...",,Incorrect information on credit report,,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",IL,60712,Closed with non-monetary relief,No


In [23]:
# test.to_csv("test_ap.csv",index=False)
# train.to_csv("train_ap.csv",index=False)


In [ ]:
loading the sparse array into memory

# print(np.load('train.npy',allow_pickle=True))

#doing transformer based UNIVERSAL SENTENCE ENCODING on narrative text based feature

#https://arxiv.org/abs/1803.11175


train=pd.read_csv("train_ap.csv",on_bad_lines='skip')
test=pd.read_csv("test_ap.csv",on_bad_lines='skip')

#fill in missing values
train=train['narrative'].fillna(" ")
test=test['narrative'].fillna(" ")

#applying encoding
embed_narrative_1 = embed(train)
embed_narrative_2 = embed(test)

#saving 512 dimensional embedding into disk
np.save("train.npy",embed_narrative_1)
np.save("test.npy",embed_narrative_2)


2023-03-26 05:36:45.304074: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-26 05:36:45.304359: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [ ]:
preprocess_2(train,"train.npy")

In [32]:
np.save("y_test.npy",np.array(binarize_target(test['disputed']).tolist()))

In [ ]:
print(np.load('train.npy',allow_pickle=True))

In [ ]:
print(np.load('test.npy',allow_pickle=True))

In [49]:
np.save("final_Test_text.npy",np.concatenate(np.concatenate((np.load("test_text_1.npy"),np.load("test_text_2.npy"),np.load("test_text_3.npy")),axis=0)))

In [46]:
np.concatenate((np.load("test_text_1.npy"),np.load("test_text_2.npy"),np.load("test_text_3.npy")),axis=0).shape

(253591, 512)